In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import pandas as pd

# Load the dataset
data = pd.read_csv('WorkNMentalHealth.csv')

# Display the first few rows of the dataset to understand its structure
data.head()

# Drop the Employee_ID column since it's not relevant for prediction
data_clean = data.drop('Employee_ID', axis=1)

# Handle missing values (if any) by filling with mode for categorical and median for numeric
for column in data_clean.select_dtypes(include=['object']).columns:
    data_clean[column].fillna(data_clean[column].mode()[0], inplace=True)
for column in data_clean.select_dtypes(include=['number']).columns:
    data_clean[column].fillna(data_clean[column].median(), inplace=True)

# Encode categorical variables using LabelEncoder
label_encoders = {}
for column in data_clean.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_clean[column] = le.fit_transform(data_clean[column])
    label_encoders[column] = le

# Separate features and target variable
X = data_clean.drop('Stress_Level', axis=1)
y = data_clean['Stress_Level']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
classification_rep = classification_report(y_test, y_pred)

# Get feature importance
feature_importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)


# Displaying the top 10 important features for simplicity
top_features = feature_importance_df.head(10)

# Show the classification report and top features
top_features

,Feature,Importance
6,Hours_Worked_Per_Week,0.104873
0,Age,0.102625
4,Years_of_Experience,0.101018
7,Number_of_Virtual_Meetings,0.084885
3,Industry,0.063249
2,Job_Role,0.063068
17,Region,0.057619
14,Company_Support_for_Remote_Work,0.055194
12,Social_Isolation_Rating,0.052952
8,Work_Life_Balance_Rating,0.050956
